In [84]:
import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd
import altair as alt
import plotly.express as px
import pydeck as pdk
import time
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras # libreria de tensorflow
from tensorflow.keras import layers # Capas de las redes neuronales
from sklearn.metrics import f1_score, classification_report
import tensorflow as tf
from xgboost import XGBClassifier
import joblib
import numpy as np


In [43]:
ruta_train = './data/train_sin_nulos.csv'
ruta_test = './data/test_sin_nulos.csv'
df_train = pd.read_csv(ruta_train, sep = ',')
df_test = pd.read_csv(ruta_test, sep = ',')

In [44]:
df_train.head()

,trip_id,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,plan_duration,...,start_year,start_hour,start_minute,start_second,end_day,end_month,end_year,end_hour,end_minute,end_second
0,101750280,35,2018-08-07 11:20:00,2018-08-07 11:55:00,33.748920,-118.275192,33.748920,-118.275192,6530,1.0,...,2018,11,20,0,7,8,2018,11,55,0
1,46560345,32,2017-09-17 17:51:00,2017-09-17 18:23:00,34.035679,-118.270813,34.047749,-118.243172,6683,0.0,...,2017,17,51,0,17,9,2017,18,23,0
2,120016336,6,2019-04-22 09:22:00,2019-04-22 09:28:00,34.046070,-118.233093,34.047749,-118.243172,6710,30.0,...,2019,9,22,0,22,4,2019,9,28,0
3,129547190,138,2019-09-22 11:27:00,2019-09-22 13:45:00,34.062580,-118.290092,34.059689,-118.294662,17068,1.0,...,2019,11,27,0,22,9,2019,13,45,0
4,136619463,14,2020-01-31 17:11:00,2020-01-31 17:25:00,34.026291,-118.277687,34.021660,-118.278687,18841,30.0,...,2020,17,11,0,31,1,2020,17,25,0


### Borrando variables creadas por defecto

In [45]:
if 'Unnamed: 0' in df_train.columns:
    df_train = df_train.drop(columns=['Unnamed: 0'])
if 'Unnamed: 0' in df_test.columns:
    df_test = df_test.drop(columns=['Unnamed: 0'])

### Variable Objetivo

Hay 6 categorías

In [46]:
df_train['passholder_type'].unique()

array(['Walk-up', 'Monthly Pass', 'One Day Pass', 'Annual Pass',
       'Flex Pass', 'Testing'], dtype=object)

### Tipo de datos

In [47]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675626 entries, 0 to 675625
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   trip_id              675626 non-null  int64  
 1   duration             675626 non-null  int64  
 2   start_time           675626 non-null  object 
 3   end_time             675626 non-null  object 
 4   start_lat            675626 non-null  float64
 5   start_lon            675626 non-null  float64
 6   end_lat              675626 non-null  float64
 7   end_lon              675626 non-null  float64
 8   bike_id              675626 non-null  int64  
 9   plan_duration        675626 non-null  float64
 10  trip_route_category  675626 non-null  object 
 11  passholder_type      675626 non-null  object 
 12  start_station        675626 non-null  int64  
 13  end_station          675626 non-null  int64  
 14  start_day            675626 non-null  int64  
 15  start_month      

In [48]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552421 entries, 0 to 552420
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   trip_id              552421 non-null  int64  
 1   duration             552421 non-null  int64  
 2   start_time           552421 non-null  object 
 3   end_time             552421 non-null  object 
 4   start_lat            552421 non-null  float64
 5   start_lon            552421 non-null  float64
 6   end_lat              552421 non-null  float64
 7   end_lon              552421 non-null  float64
 8   bike_id              552421 non-null  int64  
 9   trip_route_category  552421 non-null  object 
 10  start_station        552421 non-null  int64  
 11  end_station          552421 non-null  int64  
 12  passholder_type      552421 non-null  object 
 13  start_day            552421 non-null  int64  
 14  start_month          552421 non-null  int64  
 15  start_year       

### Seleccionando variables numéricas

In [49]:
var_num = df_train.select_dtypes(['float64', 'int64'])
var_num = var_num.drop(['trip_id'], axis = 1)
                                 

### Observando correlación entre las variables numéricas

Seleccionando las columnas con una correlación arriba de 0.8

In [50]:
# Compute the correlation matrix
correlation_matrix = var_num.corr()

# Select correlations above 0.8
correlation_threshold = 0.8

# Get the pairs of columns with correlation > 0.8
correlation_pairs = (
    correlation_matrix.where(lambda x: abs(x) > correlation_threshold)
    .stack()  # Convert to a long format
    .reset_index()  # Reset the index
)

# Filter out self-correlations (e.g., A with A)
correlation_pairs = correlation_pairs[correlation_pairs['level_0'] != correlation_pairs['level_1']]

# Rename columns for clarity
correlation_pairs.columns = ['Column 1', 'Column 2', 'Correlation']

correlation_pairs

,Column 1,Column 2,Correlation
2,start_lat,start_lon,0.982454
3,start_lon,start_lat,0.982454
6,end_lat,end_lon,0.988258
7,end_lon,end_lat,0.988258
12,start_station,end_station,0.813850
13,end_station,start_station,0.813850
16,start_day,end_day,0.997229
18,start_month,end_month,0.999655
20,start_year,end_year,0.999985
22,start_hour,end_hour,0.907298


Se borran las columnas en "Columna 1" de la lista anterior dado que la correlación es mayor a 0.8

In [51]:
columnas_a_borrar = ['start_lat', 'end_lat', 'start_station', 
                     'start_day', 'start_month', 'start_year',
                     'start_hour']
columnas_a_borrar

['start_lat',
 'end_lat',
 'start_station',
 'start_day',
 'start_month',
 'start_year',
 'start_hour']

In [52]:
df_train = df_train.drop(columnas_a_borrar, axis = 1)
df_test = df_test.drop(columnas_a_borrar, axis = 1)

In [53]:
df_train.head()

,trip_id,duration,start_time,end_time,start_lon,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,end_station,start_minute,start_second,end_day,end_month,end_year,end_hour,end_minute,end_second
0,101750280,35,2018-08-07 11:20:00,2018-08-07 11:55:00,-118.275192,-118.275192,6530,1.0,Round Trip,Walk-up,4127,20,0,7,8,2018,11,55,0
1,46560345,32,2017-09-17 17:51:00,2017-09-17 18:23:00,-118.270813,-118.243172,6683,0.0,One Way,Walk-up,3062,51,0,17,9,2017,18,23,0
2,120016336,6,2019-04-22 09:22:00,2019-04-22 09:28:00,-118.233093,-118.243172,6710,30.0,One Way,Monthly Pass,3062,22,0,22,4,2019,9,28,0
3,129547190,138,2019-09-22 11:27:00,2019-09-22 13:45:00,-118.290092,-118.294662,17068,1.0,One Way,One Day Pass,4311,27,0,22,9,2019,13,45,0
4,136619463,14,2020-01-31 17:11:00,2020-01-31 17:25:00,-118.277687,-118.278687,18841,30.0,One Way,Monthly Pass,4443,11,0,31,1,2020,17,25,0


In [54]:
for columna in df_train.columns:
    print(columna, ':', len(df_train[columna].value_counts()))

trip_id : 675626
duration : 1410
start_time : 551401
end_time : 531310
start_lon : 434
end_lon : 439
bike_id : 3351
plan_duration : 6
trip_route_category : 2
passholder_type : 6
end_station : 366
start_minute : 60
start_second : 60
end_day : 31
end_month : 12
end_year : 7
end_hour : 24
end_minute : 60
end_second : 60


### Borrando la columna trip_id

In [55]:
df_train.drop('trip_id', axis = 1, inplace=True)
df_test.drop('trip_id', axis = 1, inplace=True)

### Borrando las columnas start_time y end_time dado que ya están desagregadas

In [56]:
df_train.drop(['start_time', 'end_time'], axis = 1, inplace=True)
df_test.drop(['start_time', 'end_time'], axis = 1, inplace=True)

In [57]:
df_train.head()

,duration,start_lon,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,end_station,start_minute,start_second,end_day,end_month,end_year,end_hour,end_minute,end_second
0,35,-118.275192,-118.275192,6530,1.0,Round Trip,Walk-up,4127,20,0,7,8,2018,11,55,0
1,32,-118.270813,-118.243172,6683,0.0,One Way,Walk-up,3062,51,0,17,9,2017,18,23,0
2,6,-118.233093,-118.243172,6710,30.0,One Way,Monthly Pass,3062,22,0,22,4,2019,9,28,0
3,138,-118.290092,-118.294662,17068,1.0,One Way,One Day Pass,4311,27,0,22,9,2019,13,45,0
4,14,-118.277687,-118.278687,18841,30.0,One Way,Monthly Pass,4443,11,0,31,1,2020,17,25,0


Se elimina la variable plan_duration dado que no aparece en el dataset "test".

In [58]:
df_train.drop('plan_duration', axis = 1, inplace=True)

In [59]:
for columna in df_train.columns:
    print(columna, ':', len(df_train[columna].value_counts()))

duration : 1410
start_lon : 434
end_lon : 439
bike_id : 3351
trip_route_category : 2
passholder_type : 6
end_station : 366
start_minute : 60
start_second : 60
end_day : 31
end_month : 12
end_year : 7
end_hour : 24
end_minute : 60
end_second : 60


In [60]:
for columna in df_test.columns:
    print(columna, ':', len(df_train[columna].value_counts()))

duration : 1410
start_lon : 434
end_lon : 439
bike_id : 3351
trip_route_category : 2
end_station : 366
passholder_type : 6
start_minute : 60
start_second : 60
end_day : 31
end_month : 12
end_year : 7
end_hour : 24
end_minute : 60
end_second : 60


### Se construye un primer modelo usando dichas variables

Codificación de las variables categoricas

In [61]:
print(df_train['trip_route_category'].value_counts())
print(df_train['passholder_type'].value_counts())

trip_route_category
One Way       560051
Round Trip    115575
Name: count, dtype: int64
passholder_type
Monthly Pass    387435
Walk-up         203867
One Day Pass     39888
Annual Pass      32973
Flex Pass        11420
Testing             43
Name: count, dtype: int64


In [62]:
label_encoder = LabelEncoder()

In [63]:
df_train['trip_route_category'] = label_encoder.fit_transform(
    df_train['trip_route_category']
)
df_test['trip_route_category'] = label_encoder.fit_transform(
    df_test['trip_route_category']
)
df_train['passholder_type'] = label_encoder.fit_transform(
    df_train['passholder_type']
)
df_test['passholder_type'] = label_encoder.fit_transform(
    df_test['passholder_type']
)

In [64]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 675626 entries, 0 to 675625
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   duration             675626 non-null  int64  
 1   start_lon            675626 non-null  float64
 2   end_lon              675626 non-null  float64
 3   bike_id              675626 non-null  int64  
 4   trip_route_category  675626 non-null  int64  
 5   passholder_type      675626 non-null  int64  
 6   end_station          675626 non-null  int64  
 7   start_minute         675626 non-null  int64  
 8   start_second         675626 non-null  int64  
 9   end_day              675626 non-null  int64  
 10  end_month            675626 non-null  int64  
 11  end_year             675626 non-null  int64  
 12  end_hour             675626 non-null  int64  
 13  end_minute           675626 non-null  int64  
 14  end_second           675626 non-null  int64  
dtypes: float64(2), in

In [65]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552421 entries, 0 to 552420
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   duration             552421 non-null  int64  
 1   start_lon            552421 non-null  float64
 2   end_lon              552421 non-null  float64
 3   bike_id              552421 non-null  int64  
 4   trip_route_category  552421 non-null  int64  
 5   end_station          552421 non-null  int64  
 6   passholder_type      552421 non-null  int64  
 7   start_minute         552421 non-null  int64  
 8   start_second         552421 non-null  int64  
 9   end_day              552421 non-null  int64  
 10  end_month            552421 non-null  int64  
 11  end_year             552421 non-null  int64  
 12  end_hour             552421 non-null  int64  
 13  end_minute           552421 non-null  int64  
 14  end_second           552421 non-null  int64  
dtypes: float64(2), in

### Concatenando los conjuntos para crear los conjuntos de entrenamiento y testeo

In [66]:
df = pd.concat([df_test, df_train],ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1228047 entries, 0 to 1228046
Data columns (total 15 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   duration             1228047 non-null  int64  
 1   start_lon            1228047 non-null  float64
 2   end_lon              1228047 non-null  float64
 3   bike_id              1228047 non-null  int64  
 4   trip_route_category  1228047 non-null  int64  
 5   end_station          1228047 non-null  int64  
 6   passholder_type      1228047 non-null  int64  
 7   start_minute         1228047 non-null  int64  
 8   start_second         1228047 non-null  int64  
 9   end_day              1228047 non-null  int64  
 10  end_month            1228047 non-null  int64  
 11  end_year             1228047 non-null  int64  
 12  end_hour             1228047 non-null  int64  
 13  end_minute           1228047 non-null  int64  
 14  end_second           1228047 non-null  int64  
dty

In [67]:
df['passholder_type'].value_counts()

passholder_type
0    585394
2    387435
5    203867
3     39888
1     11420
4        43
Name: count, dtype: int64

In [68]:
X = df.drop('passholder_type', axis = 1)
y = df['passholder_type']

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [70]:
X_train.shape, X_test.shape

((982437, 14), (245610, 14))

Dado que el problema es de clasificación no binaria, se escoge la metrica f1 score ya que los datos están desbalanceados

In [71]:
def f1_metric(y_true, y_pred):
    y_pred = tf.argmax(y_pred, axis=1)  # Get predicted classes
    y_true = tf.argmax(y_true, axis=1)  # Assuming one-hot encoded labels
    
    # Use sklearn's f1_score to compute the metric
    f1 = tf.py_function(func=f1_score, inp=[y_true, y_pred, 'weighted'], Tout=tf.float32)
    return f1

### Creando una red neuronal

In [72]:
model = keras.Sequential([  
    layers.Dense(256, # cantidad de nodos de una capa 
                 activation = 'relu', # función de activación 
                 input_shape = (14,)), 
    layers.Dense(256,
                 activation = 'relu'
                 ),
    layers.Dense(6, 
                 activation='softmax') # arroja las probabilidades para cada valor numérico
])

c:\Users\Michael_PC\OneDrive - Ministerio de Ambiente y Desarrollo Sostenible\Documentos\postulaciones\arkon_data\arkon_data_test\arkon_data_env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [73]:
model.compile(optimizer = "rmsprop",
              loss = "sparse_categorical_crossentropy",
              metrics = ['accuracy'])

In [74]:
model.fit(X_train, y_train, epochs=5, batch_size=512,
          callbacks = [
                        keras.callbacks.ModelCheckpoint(
                        filepath = "mlp_arkon.keras",
                        save_best_only = True,
                        monitor = "val_loss",
                        verbose = 1)],
                        validation_data = (X_test, y_test))

Epoch 1/5
1919/1919 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3653 - loss: 118.6574
Epoch 1: val_loss improved from inf to 1.14617, saving model to mlp_arkon.keras
1919/1919 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.3654 - loss: 118.6161 - val_accuracy: 0.4762 - val_loss: 1.1462
Epoch 2/5
1915/1919 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4688 - loss: 1.1237
Epoch 2: val_loss improved from 1.14617 to 1.08369, saving model to mlp_arkon.keras
1919/1919 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.4688 - loss: 1.1237 - val_accuracy: 0.4646 - val_loss: 1.0837
Epoch 3/5
1913/1919 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4725 - loss: 1.0947
Epoch 3: val_loss did not improve from 1.08369
1919/1919 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.4725 - loss: 1.0947 - val_accuracy: 0.4620 - val_loss: 1.1203
Epoch 4/5
1918/1919 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4718 - loss: 1.0898
Epoch 4: val_loss improved from 1.08369 to 1.08011, saving model to mlp_arkon

In [75]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_loss, test_acc)

7676/7676 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.4773 - loss: 1.0754
1.0765560865402222 0.47544482350349426


### Probando el modelo Xgboost 

In [76]:
# Initialize the model
xgb_model = XGBClassifier(objective='multi:softmax', num_class=6, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=6,
              num_parallel_tree=None, ...)

In [77]:
joblib.dump(model, "xgb_model.pkl")

['xgb_model.pkl']

In [78]:
# Make predictions
y_pred = xgb_model.predict(X_test)

In [85]:
np.savetxt("./data/resultados.csv", y_pred, delimiter=",", fmt="%d")

In [83]:
len(y_pred)

245610

In [81]:
metricas = classification_report(y_test, y_pred)
print(type(metricas))
# Evaluate
print("Classification Report:\n", classification_report(y_test, y_pred))


<class 'str'>
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.76      0.59    116956
           1       0.43      0.01      0.02      2252
           2       0.50      0.30      0.37     77685
           3       0.34      0.01      0.02      7897
           4       0.00      0.00      0.00         6
           5       0.48      0.20      0.28     40814

    accuracy                           0.49    245610
   macro avg       0.37      0.21      0.21    245610
weighted avg       0.48      0.49      0.45    245610

